In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data

In [2]:
def load_financial_data(name, output_file):
    try:
        df = pd.read_pickle(output_file)
        print('File data found...reading',name,'data')
    except FileNotFoundError:
        print('File not found...downloading', name, 'data')
        df = data.DataReader(name, 'yahoo', '2001-01-01', '2018-01-01')
        df.to_pickle(output_file)
    return df

In [3]:
SP500 = load_financial_data('^GSPC', '^GSPC_data.pkl')

File not found...downloading ^GSPC data
